In [2]:
import pandas as pd 

df = pd.read_csv('100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
# tokenize 
def tokenize(text):
    # preprocess
    text = text.lower()

    text = text.replace('?','')

    text = text.replace("'","")
    
    return text.split()

In [4]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [5]:
# vocab form
vocab = {'<UNK>':0}

In [6]:
def build_vocab(row):
    tokenized_question = tokenize(row['question'])
    tokenized_answer = tokenize(row['answer'])

    merged_tokens = tokenized_question + tokenized_answer

    for token in merged_tokens:
        if token not in vocab:
            vocab[token] = len(vocab)

In [7]:
df.apply(build_vocab,axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [8]:
len(vocab)

324

In [9]:
# convert words to numerical Indices

def text_to_indices(text,vocab):

    indexed_text = []

    for token in tokenize(text):

        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab['<UNK>'])

    return indexed_text

In [10]:
text_to_indices("what is compusx",vocab)

[1, 2, 0]

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

In [12]:
class QADataset(Dataset):
    def __init__(self,df,vocab):
        self.df = df 
        self.vocab = vocab
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self,index):
        numerical_question = text_to_indices(self.df.iloc[index]['question'],self.vocab)
        numerical_answer = text_to_indices(self.df.iloc[index]['answer'],self.vocab)

        return torch.tensor(numerical_question),torch.tensor(numerical_answer)

In [13]:
dataset = QADataset(df,vocab)

In [14]:
dataloader = DataLoader(dataset,batch_size=1,shuffle=True)

In [15]:
for question,answer in dataloader:
    print(question,answer)

tensor([[ 42, 290, 291, 118, 292, 158, 293, 294]]) tensor([[295]])
tensor([[ 78,  79, 261, 151,  14, 262, 153]]) tensor([[36]])
tensor([[ 42, 167,   2,   3,  17, 168, 169]]) tensor([[170]])
tensor([[ 42, 137,   2,  62,  39,   3, 322, 323]]) tensor([[6]])
tensor([[ 42, 263, 264,  14, 265, 266, 158, 267]]) tensor([[268]])
tensor([[ 10, 140,   3, 141, 171,   5,   3,  70, 172]]) tensor([[173]])
tensor([[ 42, 137,   2, 138,  39, 175, 269]]) tensor([[99]])
tensor([[ 42, 107,   2, 108,  19, 109]]) tensor([[110]])
tensor([[  1,   2,   3,   4,   5, 286]]) tensor([[287]])
tensor([[ 10, 308,   3, 309, 310]]) tensor([[311]])
tensor([[ 1,  2,  3, 69,  5,  3, 70, 71]]) tensor([[72]])
tensor([[  1,   2,   3,  17, 115,  83,  84]]) tensor([[116]])
tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]]) tensor([[28]])
tensor([[  1,   2,   3,  37, 133,   5,  26]]) tensor([[134]])
tensor([[1, 2, 3, 4, 5, 6]]) tensor([[7]])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([[68]])
tensor([[  1,   2,   3, 146, 147,  19, 148

In [16]:
import torch.nn as nn 

In [17]:
dataset[0][0]

tensor([1, 2, 3, 4, 5, 6])

In [18]:
x = nn.Embedding(324,embedding_dim=50)

In [19]:
a = x(dataset[0][0])

In [20]:
y = nn.RNN(50,64)

In [21]:
y(a)[0].shape

torch.Size([6, 64])

In [22]:
y(a)[1].shape

torch.Size([1, 64])

In [23]:
# Hidden state
y(a)[0].shape

torch.Size([6, 64])

In [24]:
# final output O6
b = y(a)[1]
b.shape

torch.Size([1, 64])

In [25]:
z = nn.Linear(64,324)

In [26]:
z(b).shape

torch.Size([1, 324])

In [27]:
class SimpleRNN(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
        self.rnn = nn.RNN(50,64,batch_first=True)
        self.fc = nn.Linear(64,vocab_size)

    def forward(self,question):
        embedded_question = self.embedding(question)
        hidden, final = self.rnn(embedded_question)
        output = self.fc(final.squeeze(0))

        return output

In [35]:
learning_rate = 0.001
epochs = 50

In [36]:
model = SimpleRNN(len(vocab))

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)


In [38]:
# training loop

for epoch in range(epochs):

    total_loss = 0

    for question, answer in dataloader:

        optimizer.zero_grad()

        #forward pass
        output = model(question)

        # loss -> output shape (1,324) - answer(1)
        loss = criterion(output,answer[0]) # answer in 2d 

        # gradients
        loss.backward()

        # update
        optimizer.step()

        total_loss = total_loss + loss.item()
    print(f"Epoch {epoch+1} loss: {total_loss:4f}")


Epoch 1 loss: 526.839797
Epoch 2 loss: 454.667216
Epoch 3 loss: 373.803793
Epoch 4 loss: 314.571455
Epoch 5 loss: 263.864618
Epoch 6 loss: 215.950577
Epoch 7 loss: 172.451946
Epoch 8 loss: 134.318796
Epoch 9 loss: 102.921423
Epoch 10 loss: 78.887479
Epoch 11 loss: 60.858163
Epoch 12 loss: 47.591721
Epoch 13 loss: 38.033226
Epoch 14 loss: 30.870940
Epoch 15 loss: 25.581594
Epoch 16 loss: 21.517505
Epoch 17 loss: 18.283601
Epoch 18 loss: 15.490217
Epoch 19 loss: 13.404168
Epoch 20 loss: 11.594411
Epoch 21 loss: 10.097400
Epoch 22 loss: 8.862629
Epoch 23 loss: 7.824874
Epoch 24 loss: 6.955554
Epoch 25 loss: 6.182242
Epoch 26 loss: 5.568179
Epoch 27 loss: 5.023323
Epoch 28 loss: 4.551226
Epoch 29 loss: 4.141996
Epoch 30 loss: 3.777503
Epoch 31 loss: 3.464157
Epoch 32 loss: 3.185632
Epoch 33 loss: 2.943147
Epoch 34 loss: 2.711675
Epoch 35 loss: 2.512952
Epoch 36 loss: 2.338771
Epoch 37 loss: 2.169508
Epoch 38 loss: 2.022240
Epoch 39 loss: 1.885024
Epoch 40 loss: 1.759392
Epoch 41 loss: 1.64

In [39]:
def predict(model,question,threshold=0.5):

    # convert to numbers
    numerical_question = text_to_indices(question,vocab)
    

    # tensor
    question_tensor = torch.tensor(numerical_question).unsqueeze(0)

    # sent to model
    output = model(question_tensor)

    # convert logits to probs
    probs = torch.nn.functional.softmax(output,dim=1)

    # find index of max prob
    value , index = torch.max(probs,dim=1)

    
    if value < threshold:
        print("I don't Know")
    else:
        print(list(vocab.keys())[index])


In [40]:
predict(model,"what is the largest planet in our solar system")

jupiter


In [45]:
predict(model,"what is the capital of India")

delhi
